In [12]:
import sys
from pathlib import Path

# Notebook is in verf/ttv → parent is verf
VERF_DIR = Path.cwd().parent
sys.path.insert(0, str(VERF_DIR))

print("Project root added to sys.path:", VERF_DIR)

Project root added to sys.path: /home/adhyeta/0zProject/verf


In [13]:
VERF_DIR

PosixPath('/home/adhyeta/0zProject/verf')

In [14]:
import os
from pathlib import Path
import random
import numpy as np
import torch
import torch.nn.functional as F

import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc


In [15]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


In [17]:
from ttv.config import CFG
from ttv.models import SiameseFusion


In [ ]:

# Adjust this if your notebook is not in verf/
DATA_DIR = VERF_DIR / "processed_tensors"
TEST_IDS_PATH = VERF_DIR / "ttv" / "test_subjects-251214_1500.txt"
# Checkpoint path (edit)
CKPT_PATH = VERF_DIR / "ttv" / "best_gait_model-251214_1500.pth"  # example
DATA_DIR,TEST_IDS_PATH,CKPT_PATH

(PosixPath('/home/adhyeta/0zProject/verf/processed_tensors'),
 PosixPath('/home/adhyeta/0zProject/verf/test_subjects-251214_1500.txt'),
 PosixPath('/home/adhyeta/0zProject/verf/best_gait_model-251214_1500.pth'))

In [ ]:

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE


In [6]:
test_ids = [line.strip() for line in TEST_IDS_PATH.read_text().splitlines() if line.strip()]
len(test_ids), test_ids[:5]

(14, ['S023', 'S012', 'S173', 'S035', 'S028'])

In [9]:
# Update these imports to your current package layout
from config import CFG
from models import SiameseFusion

model = SiameseFusion(cfg=CFG).to(DEVICE)
model.eval()

raw = torch.load(str(CKPT_PATH), map_location=DEVICE)
state_dict = raw["model_state_dict"] if isinstance(raw, dict) and "model_state_dict" in raw else raw

# Optional: handle older key rename .cnn -> .net (only if needed)
if any(".cnn." in k for k in state_dict.keys()):
    state_dict = {k.replace(".cnn.", ".net."): v for k, v in state_dict.items()}

missing, unexpected = model.load_state_dict(state_dict, strict=False)
print("Missing keys:", len(missing))
print("Unexpected keys:", len(unexpected))


ImportError: attempted relative import with no known parent package